In [ ]:
# This notebook attempts to synchronize one TAS up with another after a certain point.
# In more detail, suppose TAS T1 gets to a certain situation on frame X and TAS T2 gets
# to a similar situation on frame Y. We wish to have B mimic TAS A from frame X to
# some larger frame number Z.

# To accomplish this, the program essentially appends T1[X:Z] to T2[:Y], making a tas T3 of length
# Y+(Z-X). The program then tweaks the joystick values. To determine which tweaks to use, it checks a fitness
# function based on variables provided by the user. The user can provide a set of variables
# they wish to match exactly, a set of variables they wish to match within some tolerance,
# and a set of variables they wish to penalize with some weight.

# It optimizes joystick sections leading up to each frame of T3 from Y to Y+Z-X one at a time. It
# keeps the best solution according to the provided fitness function and advances a frame at a time.

In [1]:
from typing import *
import wafel
import ctypes as C
import struct
from random import *
import numpy as np
import copy
import numpy as np

In [2]:
%run utils.ipynb

In [3]:
def get_fitness(game,
                game_vars_match_exact, game_vars_match_range, game_vars_approximate_weighted):
    # Computes a fitness function based on what is provided by the user.
    fitness = 0
    success = True
    for var in game_vars_match_exact:
        # If we are wrong on an exact variable, apply an extremely large penalty.
        if game.read(var) != game_vars_match_exact[var]:
            #note: if your weights/possible changes in values for the weighted
            #approximate variables are pretty small, you may need to
            #reduce this penalty so those changes aren't lost in float rounding if you
            #care about other changes even when one of the exact matches fails.
            fitness += 1e10
    for var in game_vars_match_range:
        val = game.read(var)
        goal = game_vars_match_range[var][0]
        tol = game_vars_match_range[var][1]
        # If we are outside of the tolerance, apply a very large penalty proportional to how far we are.
        # Ensure that it doesn't exceed the importance of an exact variable, though.
        if abs(val - goal) > tol:
            fitness += min(1e6*(abs(val - goal) - tol)/tol, 1e9)
    for var in game_vars_approximate_weighted:
        val = game.read(var)
        fitness += abs(val - game_vars_approximate_weighted[var][0])*game_vars_approximate_weighted[var][1]
    return fitness

In [4]:
def l1_of_diff(m64):
    '''Regularization in the input joystick for smoother inputs'''
    tot = 0
    for i in range(len(m64) - 1):
        tot += abs(m64[i].stick_x - m64[i+1].stick_x) + abs(m64[i].stick_y - m64[i+1].stick_y)
    return tot

#Special version of the annealing function for resyncing.
#Has some bells and whistles.
def anneal_for_resync(game,
                      m64,
                      starting_savestate,
                      start_frame, end_frame,
                      game_vars_match_exact, game_vars_match_range, game_vars_approximate_weighted,
                      weighted_sum_acceptable,
                      max_iterations,
                      starting_temperature,
                      l1_diff_penalty_strength = 0.0):
    '''Optimizes joystick inputs one frame at a time to make a TAS match up with desired values.
    game is a Wafel game, where starting_savestate is a savestate formed at start_frame of m64.
    The program optimizes joystick inputs from start_frame to end_frame
    such that a fitness function based on game_vars_match_exact, game_vars_match_range, and
    game_vars_approximate_weighted is minimized at end_frame. Each of these three is a dictonary
    where game_vars_match_exact maps Wafel variables to desired values and game_vars_match_range and
    game_vars_approximate_weighted map Wafel variables to tuples (desired value, max distance) and
    (desired_value, coefficient) respectively.
    weighted_sum_acceptable specifies a value of the fitness function below which the optimizer should stop
    to save time and move on to the next frame.
    max_iterations gives a maximum number of perturbations to try.
    starting_temperature specifies the initial temperature of the annealing.
    l1_diff_penalty_strength, if positive, imposes a penalty on jittery joystick inputs.'''
    game.load_state(starting_savestate)
    
    
    for f in range(start_frame, end_frame):
        set_inputs(game, m64[f])
        game.advance()
    
    # Since the camera may be different, the newest
    # frame to sync could be way off from what we want.
    # Therefore, start by doing a search of all possible joysticks for the very final frame.
    # Takes a long time, so I go in coarser intervals. Annealing should be
    # able to find its way into a better granularity.
    best_fitness = 1e15
    best_input = None
    before_final_step = game.save_state()
    for x in range(0, 256, 4):
        for y in range(0, 256, 4):
            game.load_state(before_final_step)
            new_input = wafel.Input(buttons = m64[end_frame].buttons, stick_x = x, stick_y = y)
            set_inputs(game, new_input)
            game.advance()
            fitness = get_fitness(game, game_vars_match_exact, game_vars_match_range, game_vars_approximate_weighted)
            if l1_diff_penalty_strength > 0:
                fitness += l1_diff_penalty_strength*l1_of_diff(m64[start_frame:end_frame+1])
            if fitness < best_fitness:
                best_fitness = fitness
                best_input = new_input
    m64[end_frame] = best_input
    
    
    print('*************************************************************')
    print('BEFORE ANNEALING')
    print('Frame {0}: fitness is {1}'.format(end_frame, best_fitness))
    game.load_state(before_final_step)
    set_inputs(game, best_input)
    game.advance()
    print('Exact matches:')
    for var in game_vars_match_exact:
        print('{0}: wanted {1}, got {2}'.format(var, game_vars_match_exact[var], game.read(var)))
    print('Matches within an interval:')
    for var in game_vars_match_range:
        print('{0}: wanted {1}, tolerance {2}, got {3}'.format(var, game_vars_match_range[var][0],
                                                               game_vars_match_range[var][1], game.read(var)))
    print('Weighted approximations:')
    for var in game_vars_approximate_weighted:
        print('{0}: wanted {1}, weight {2}, got {3}'.format(var, game_vars_approximate_weighted[var][0],
                                                            game_vars_approximate_weighted[var][1], game.read(var)))
        
    
    
    # Now do the annealing step
    # Not going to bother with random restarts here, though it is a plausible improvement.
    # Probably better for now to just go with very low temperature since we need to do faster
    # searches. Therefore default starting temperature is 0. Plausible to use nonzero ones but
    # they should be pretty small unless you're willing to wait a long time or you want to
    # do random restarts.
    orig_m64 = copy.copy(m64)
    best_m64 = copy.copy(m64)
    
    cur_fitness = best_fitness
    
    # Bunch of parameters here to play with
    # Might want to make them function arguments at some point
    many_change_prob = .1
    big_change_prob = .3
    max_changes = 5
    max_size = 30
    
    # Weight the distribution to preferentially choose the frames toward the end
    # I'm guessing this will help at least somewhat, since the earlier frames should already have
    # been in a good position.
    possible_frames = np.arange(start_frame, end_frame + 1)
    exp_decay = np.exp(-np.arange(end_frame - start_frame, -1, -1)*(1.0/(start_frame-end_frame+1)))
    exp_decay = exp_decay/(exp_decay.sum())
    
    temp = starting_temperature
    
    # Typical annealing optimizer
    for it in range(max_iterations):
        # Terminate if we have already reached an acceptable fitness
        if cur_fitness <= weighted_sum_acceptable:
            break
        if it % 300 == 0:
            temp *= .97
        changed_frames = []
        num_changes = randint(1, max_changes)
        if random() > many_change_prob:
            num_changes = 1
        change_frame_nums = np.random.choice(possible_frames, size = num_changes, p = exp_decay)
        for change_frame_num in change_frame_nums:
            changed_frames.append((change_frame_num, m64[change_frame_num]))
            change_size = randint(1, max_size)
            if random() > big_change_prob:
                change_size = 1
            change_dir = randint(0, 3)
            frame_inp = m64[change_frame_num]
            new_inp = wafel.Input(frame_inp.buttons, frame_inp.stick_x, frame_inp.stick_y)
            m64[change_frame_num] = new_inp
            for inc in range(change_size):
                if change_dir == 0:
                    new_inp.stick_x = increment_coord(new_inp.stick_x)
                elif change_dir == 1:
                    new_inp.stick_y = increment_coord(new_inp.stick_y)
                elif change_dir == 2:
                    new_inp.stick_x = 255 - increment_coord(255 - new_inp.stick_x)
                else:
                    new_inp.stick_y = 255 - increment_coord(255 - new_inp.stick_y)
        game.load_state(starting_savestate)
        for frame in range(start_frame, end_frame + 1):
            set_inputs(game, m64[frame])
            game.advance()
        fitness = get_fitness(game, game_vars_match_exact, game_vars_match_range, game_vars_approximate_weighted)
        if l1_diff_penalty_strength > 0:
            fitness += l1_diff_penalty_strength*l1_of_diff(m64[start_frame:end_frame+1])
        
        if fitness < cur_fitness:
            cur_fitness = fitness
            if fitness < best_fitness:
                best_fitness = fitness
                best_m64 = copy.copy(m64)
        elif temp > 0 and random() < np.exp(-(fitness - cur_fitness) / temp):
            cur_fitness = fitness
        else:
            for changed_frame_info in changed_frames[::-1]:
                m64[changed_frame_info[0]] = changed_frame_info[1]
    
    for i in range(len(best_m64)):
        m64[i] = best_m64[i]
    game.load_state(starting_savestate)
    for frame in range(start_frame, end_frame + 1):
        set_inputs(game, m64[frame])
        game.advance()
    
    print('*************************************************************')
    print('AFTER ANNEALING')
    print('Frame {0}: fitness is {1}'.format(end_frame, best_fitness))
    print('Exact matches:')
    for var in game_vars_match_exact:
        print('{0}: wanted {1}, got {2}'.format(var, game_vars_match_exact[var], game.read(var)))
    print('Matches within an interval:')
    for var in game_vars_match_range:
        print('{0}: wanted {1}, tolerance {2}, got {3}'.format(var, game_vars_match_range[var][0],
                                                               game_vars_match_range[var][1], game.read(var)))
    print('Weighted approximations:')
    for var in game_vars_approximate_weighted:
        print('{0}: wanted {1}, weight {2}, got {3}'.format(var, game_vars_approximate_weighted[var][0],
                                                            game_vars_approximate_weighted[var][1], game.read(var)))
    
    
    
    return best_fitness

In [5]:

def resync_tas(game,
               filename_original, filename_toresync, filename_output,
               start_frame_original, end_frame_original, start_frame_toresync,
               game_vars_match_exact = None,
               game_vars_match_range = None,
               game_vars_approximate_weighted = None,
               weighted_sum_acceptable = 0.0,
               frames_back = 20,
               max_iterations = 3000,
               starting_temperature = 0.0,
               l1_diff_penalty_strength = 0.0):
    '''Tries to sync up filename_toresync to filename_original and saves the result to output_filename
For each variable of interest, it tries to find joystick values in filename_new that
will match up variables.
game_vars_match_exact, if not None, is a list of variable strings which
the function will try to sync exactly.
game_vars_match_range, if not None, is a dictionary from variable strings
to tolerances which define the amount by which that variable can be off
game_vars_approximate_weighted, if not None, is a dictionary from variable strings
to weights. The function will attempt to minimize
sum(abs(original_value-new_value)*weight) given that the matching variables
all have acceptable values.
It is permissible for variables to apear both in the weighted sum and in the matching.
This may be desirable if e.g. it's necessary to have yaw in a certain interval and we
also want to get it close to the center of that interval.
The function tries to match 1 frame, then 2, etc. If the matching variables are
all acceptable and the sum of the weighted errors is at most weighted_sum_acceptable,
the function will continue to the next frame.
Otherwise, it will try max_iteration times.
By default, joystick perturbation interval is 20 frames. Can change this with frames_back'''
    
    power_on = game.save_state()
    m64_original = wafel.load_m64(filename_original)
    m64_toresync = wafel.load_m64(filename_toresync)
    
    relevant_variables = set([var for var in game_vars_match_exact] +
                             [var for var in game_vars_match_range] +
                             [var for var in game_vars_approximate_weighted])
    # First, find values of the variables in the original TAS.
    observed_values = {var: [] for var in relevant_variables}
    # Advance to start of resync window
    for f in range(start_frame_original):
        set_inputs(game, m64_original[1][f])
        game.advance()
    # Now record values
    for f in range(end_frame_original - start_frame_original + 1):
        set_inputs(game, m64_original[1][f + start_frame_original])
        game.advance()
        for var in relevant_variables:
            observed_values[var].append(game.read(var))
            
    
    # Now start working on the output TAS.
    # Use the start of the new one and the end of the old one.
    # Then anneal to try to match values.
    m64_output = m64_toresync[1][:start_frame_toresync] + m64_original[1][start_frame_original:]
    game.load_state(power_on)
    start_frame_anneal = start_frame_toresync - frames_back
    for f in range(start_frame_anneal):
        set_inputs(game, m64_output[f])
        game.advance()
    savestate_before_initial_conditions = game.save_state()
    frames_resyncing_count = 0
    while frames_resyncing_count < end_frame_original - start_frame_original + 1:
        result = 1e10
        # Track the number of retries if the result stays ridiculously high.
        # Spend more time optimizing as the number of tries increases.
        retries = 0
        # If the fitness function is too large, keep trying.
        while result >= 20000:
            if retries > 0:
                print('RETRY {0}'.format(retries))
            m64_output_prev = copy.copy(m64_output)
            # In this call, max_iterations and starting_temperature both increase with the number of retries
            # to improve the chances of fixing the TAS.
            result = anneal_for_resync(game,
                              m64_output,
                              savestate_before_initial_conditions,
                              start_frame_anneal + frames_resyncing_count, start_frame_anneal + frames_back + frames_resyncing_count,
                              {var: observed_values[var][frames_resyncing_count] for var in game_vars_match_exact},
                              {var: (observed_values[var][frames_resyncing_count], game_vars_match_range[var]) for var in game_vars_match_range},
                              {var: (observed_values[var][frames_resyncing_count], game_vars_approximate_weighted[var]) for var in game_vars_approximate_weighted},
                              weighted_sum_acceptable = weighted_sum_acceptable,
                              max_iterations = max_iterations*((frames_resyncing_count == 0) * 10 + 1 + retries), # Take extra effort to resync the start
                              starting_temperature = starting_temperature*(5**retries),
                              l1_diff_penalty_strength = l1_diff_penalty_strength)
            retries += 1
            # We'll give up after 20 retries
            if retries > 20:
                break
        game.load_state(savestate_before_initial_conditions)
        set_inputs(game, m64_output[start_frame_anneal + frames_resyncing_count])
        game.advance()
        savestate_before_initial_conditions = game.save_state()
        frames_resyncing_count += 1
        wafel.save_m64(filename_output, m64_toresync[0], m64_output)
    

In [ ]:
# resync_tas(game,
#                filename_original, filename_toresync, filename_output,
#                start_frame_original, end_frame_original, start_frame_toresync,
#                game_vars_match_exact = None,
#                game_vars_match_range = None,
#                game_vars_approximate_weighted = None,
#                weighted_sum_acceptable = 0.0,
#                frames_back = 20,
#                max_iterations = 10000,
#                starting_temperature = 0.0):

In [ ]:
# ***********************************************************************************************
# The below are simply various calls that I've done in the past. Not necessary to read, but I
# like to keep them around to glance at or copy/paste
# ***********************************************************************************************

In [ ]:
game = wafel.Game('sm64_us.dll')
main_folder = 'm64s/'

filename_original = main_folder+'ccmreds_orig.m64'
filename_toresync = main_folder+'ccmreds_m2f.m64'
filename_output = main_folder+'ccmreds_resynced.m64'

game_vars_match_exact = ['gMarioState.action',
                         'gMarioState.numCoins']
game_vars_match_range = {'gMarioState.pos[0]': 80,
                         'gMarioState.pos[1]': 20,
                         'gMarioState.pos[2]': 80,
                         'gMarioState.forwardVel': 0.04}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 15.0,
                                  'gMarioState.faceAngle[1]': 0.3,
                                  'gMarioState.forwardVel': 80.0
                                  }


resync_tas(game,
           filename_original, filename_toresync, filename_output,
           5042, 5490, 5040,
           #4961, 5490, 4959,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 1.5,
           weighted_sum_acceptable = 0.01)


In [ ]:
# resync_tas(game,
#                filename_original, filename_toresync, filename_output,
#                start_frame_original, end_frame_original, start_frame_toresync,
#                game_vars_match_exact = None,
#                game_vars_match_range = None,
#                game_vars_approximate_weighted = None,
#                weighted_sum_acceptable = 0.0,
#                frames_back = 20,
#                max_iterations = 10000,
#                starting_temperature = 0.0):

In [11]:
game = wafel.Game('sm64_us.dll')
main_folder = 'm64s/'

filename_original = main_folder+'fight_king_twice.m64'
filename_toresync = main_folder+'fight_king_twice.m64'
filename_output = main_folder+'bob100_resynced_notext.m64'

game_vars_match_exact = ['gMarioState.action']
game_vars_match_range = {'gMarioState.pos[0]': 3.5,
                         'gMarioState.pos[1]': 40,
                         'gMarioState.pos[2]': 3.5,
                         'gObjectPool[113]->oPosX': 5.0,
                         'gObjectPool[113]->oPosZ': 5.0,
                         'gObjectPool[113]->oFaceAngleYaw': 16,
                         'gMarioState.faceAngle[1]': 128,
                         'gMarioState.forwardVel': .25}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 15.0,
                                  #'gMarioState.faceAngle[1]': 0.3,
                                  'gMarioState.forwardVel': 30.0
                                  }

resync_tas(game,
           filename_original, filename_toresync, filename_output,
           22097, 22994, 23441,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 0.02,
           weighted_sum_acceptable = 0.01,
           frames_back = 30,
                max_iterations = 5000)


*************************************************************
BEFORE ANNEALING
Frame 23441: fitness is 0.0
Exact matches:
gMarioState.action: wanted 205521409, got 205521409
Matches within an interval:
gMarioState.pos[0]: wanted -6525.123046875, tolerance 3.5, got -6525.123046875
gMarioState.pos[1]: wanted -0.0, tolerance 40, got -0.0
gMarioState.pos[2]: wanted 6431.123046875, tolerance 3.5, got 6431.123046875
gObjectPool[113]->oPosX: wanted 1636.0, tolerance 5.0, got 1636.0
gObjectPool[113]->oPosZ: wanted -5567.0, tolerance 5.0, got -5567.0
gObjectPool[113]->oFaceAngleYaw: wanted -26760, tolerance 16, got -26760
gMarioState.faceAngle[1]: wanted 24576, tolerance 128, got 24576
gMarioState.forwardVel: wanted 0.0, tolerance 0.25, got 0.0
Weighted approximations:
gMarioState.pos[0]: wanted -6525.123046875, weight 15.0, got -6525.123046875
gMarioState.pos[1]: wanted -0.0, weight 1.0, got -0.0
gMarioState.pos[2]: wanted 6431.123046875, weight 15.0, got 6431.123046875
gMarioState.forwardVel:

*************************************************************
BEFORE ANNEALING
Frame 23445: fitness is 0.029382705688476562
Exact matches:
gMarioState.action: wanted 50333824, got 50333824
Matches within an interval:
gMarioState.pos[0]: wanted -6522.318359375, tolerance 3.5, got -6522.318359375
gMarioState.pos[1]: wanted 120.68546295166016, tolerance 40, got 120.68546295166016
gMarioState.pos[2]: wanted 6468.224609375, tolerance 3.5, got 6468.2265625
gObjectPool[113]->oPosX: wanted 1636.0, tolerance 5.0, got 1636.0
gObjectPool[113]->oPosZ: wanted -5567.0, tolerance 5.0, got -5567.0
gObjectPool[113]->oFaceAngleYaw: wanted -26760, tolerance 16, got -26760
gMarioState.faceAngle[1]: wanted 814, tolerance 128, got 809
gMarioState.forwardVel: wanted 10.581157684326172, tolerance 0.25, got 10.581154823303223
Weighted approximations:
gMarioState.pos[0]: wanted -6522.318359375, weight 15.0, got -6522.318359375
gMarioState.pos[1]: wanted 120.68546295166016, weight 1.0, got 120.68546295166016
gMa

*************************************************************
BEFORE ANNEALING
Frame 23449: fitness is 0.03281593322753906
Exact matches:
gMarioState.action: wanted 50333824, got 50333824
Matches within an interval:
gMarioState.pos[0]: wanted -6518.25390625, tolerance 3.5, got -6518.25390625
gMarioState.pos[1]: wanted 225.59951782226562, tolerance 40, got 225.59951782226562
gMarioState.pos[2]: wanted 6521.892578125, tolerance 3.5, got 6521.89453125
gObjectPool[113]->oPosX: wanted 1636.0, tolerance 5.0, got 1636.0
gObjectPool[113]->oPosZ: wanted -5567.0, tolerance 5.0, got -5567.0
gObjectPool[113]->oFaceAngleYaw: wanted -26760, tolerance 16, got -26760
gMarioState.faceAngle[1]: wanted 814, tolerance 128, got 809
gMarioState.forwardVel: wanted 15.181148529052734, tolerance 0.25, got 15.181031227111816
Weighted approximations:
gMarioState.pos[0]: wanted -6518.25390625, weight 15.0, got -6518.25390625
gMarioState.pos[1]: wanted 225.59951782226562, weight 1.0, got 225.59951782226562
gMarioS

*************************************************************
BEFORE ANNEALING
Frame 23453: fitness is 0.0405120849609375
Exact matches:
gMarioState.action: wanted 50333824, got 50333824
Matches within an interval:
gMarioState.pos[0]: wanted -6512.78125, tolerance 3.5, got -6512.783203125
gMarioState.pos[1]: wanted 221.82803344726562, tolerance 40, got 221.82803344726562
gMarioState.pos[2]: wanted 6593.912109375, tolerance 3.5, got 6593.912109375
gObjectPool[113]->oPosX: wanted 1636.0, tolerance 5.0, got 1636.0
gObjectPool[113]->oPosZ: wanted -5567.0, tolerance 5.0, got -5567.0
gObjectPool[113]->oFaceAngleYaw: wanted -26760, tolerance 16, got -26760
gMarioState.faceAngle[1]: wanted 814, tolerance 128, got 809
gMarioState.forwardVel: wanted 19.781139373779297, tolerance 0.25, got 19.780765533447266
Weighted approximations:
gMarioState.pos[0]: wanted -6512.78125, weight 15.0, got -6512.783203125
gMarioState.pos[1]: wanted 221.82803344726562, weight 1.0, got 221.82803344726562
gMarioState

*************************************************************
BEFORE ANNEALING
Frame 23457: fitness is 0.3398323059082031
Exact matches:
gMarioState.action: wanted 50333824, got 50333824
Matches within an interval:
gMarioState.pos[0]: wanted -6505.8984375, tolerance 3.5, got -6505.8828125
gMarioState.pos[1]: wanted 154.05654907226562, tolerance 40, got 154.05654907226562
gMarioState.pos[2]: wanted 6684.275390625, tolerance 3.5, got 6684.26953125
gObjectPool[113]->oPosX: wanted 1636.0, tolerance 5.0, got 1636.0
gObjectPool[113]->oPosZ: wanted -5567.0, tolerance 5.0, got -5567.0
gObjectPool[113]->oFaceAngleYaw: wanted -26760, tolerance 16, got -26760
gMarioState.faceAngle[1]: wanted 814, tolerance 128, got 809
gMarioState.forwardVel: wanted 24.38113021850586, tolerance 0.25, got 24.380544662475586
Weighted approximations:
gMarioState.pos[0]: wanted -6505.8984375, weight 15.0, got -6505.8828125
gMarioState.pos[1]: wanted 154.05654907226562, weight 1.0, got 154.05654907226562
gMarioState.p

...

*************************************************************
BEFORE ANNEALING
Frame 23977: fitness is 0.44388771057128906
Exact matches:
gMarioState.action: wanted 16779394, got 16779394
Matches within an interval:
gMarioState.pos[0]: wanted 1358.2750244140625, tolerance 3.5, got 1358.2818603515625
gMarioState.pos[1]: wanted 4713.0, tolerance 40, got 4713.0
gMarioState.pos[2]: wanted -5207.5576171875, tolerance 3.5, got -5207.5576171875
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 16, got -5864
gMarioState.faceAngle[1]: wanted 147, tolerance 128, got 160
gMarioState.forwardVel: wanted -9.320931434631348, tolerance 0.25, got -9.33230972290039
Weighted approximations:
gMarioState.pos[0]: wanted 1358.2750244140625, weight 15.0, got 1358.2818603515625
gMarioState.pos[1]: wanted 4713.0, weight 1.0, got 4713.0

*************************************************************
BEFORE ANNEALING
Frame 23981: fitness is 0.6754589080810547
Exact matches:
gMarioState.action: wanted 16779394, got 16779394
Matches within an interval:
gMarioState.pos[0]: wanted 1329.6451416015625, tolerance 3.5, got 1329.6217041015625
gMarioState.pos[1]: wanted 4517.0, tolerance 40, got 4517.0
gMarioState.pos[2]: wanted -5251.7158203125, tolerance 3.5, got -5251.6943359375
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 16, got -5864
gMarioState.faceAngle[1]: wanted 147, tolerance 128, got 160
gMarioState.forwardVel: wanted -11.498113632202148, tolerance 0.25, got -11.498059272766113
Weighted approximations:
gMarioState.pos[0]: wanted 1329.6451416015625, weight 15.0, got 1329.6217041015625
gMarioState.pos[1]: wanted 4517.0, weight 1.0, got 4517

*************************************************************
AFTER ANNEALING
Frame 23984: fitness is 0.3666400909423828
Exact matches:
gMarioState.action: wanted 16779394, got 16779394
Matches within an interval:
gMarioState.pos[0]: wanted 1315.2354736328125, tolerance 3.5, got 1315.2242431640625
gMarioState.pos[1]: wanted 4328.0, tolerance 40, got 4328.0
gMarioState.pos[2]: wanted -5291.8642578125, tolerance 3.5, got -5291.8603515625
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 16, got -5864
gMarioState.faceAngle[1]: wanted 147, tolerance 128, got 160
gMarioState.forwardVel: wanted -14.438450813293457, tolerance 0.25, got -14.443103790283203
Weighted approximations:
gMarioState.pos[0]: wanted 1315.2354736328125, weight 15.0, got 1315.2242431640625
gMarioState.pos[1]: wanted 4328.0, weight 1.0, got 4328.

*************************************************************
AFTER ANNEALING
Frame 23987: fitness is 41586663.12912396
Exact matches:
gMarioState.action: wanted 67110008, got 67110008
Matches within an interval:
gMarioState.pos[0]: wanted 1298.34228515625, tolerance 3.5, got 1320.1639404296875
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5451.744140625, tolerance 3.5, got -5321.0205078125
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 16, got -5879
gMarioState.faceAngle[1]: wanted 147, tolerance 128, got 160
gMarioState.forwardVel: wanted -15.221612930297852, tolerance 0.25, got -14.990771293640137
Weighted approximations:
gMarioState.pos[0]: wanted 1298.34228515625, weight 15.0, got 1320.1639404296875
gMarioState.pos[1]: wanted 4293.0, weight 1.0, got 4293.0
gMar

KeyboardInterrupt: 

In [14]:
#add 500
game = wafel.Game('sm64_us.dll')
main_folder = 'm64s/'

filename_original = main_folder+'fight_king_twice.m64'
filename_toresync = main_folder+'fight_king_twice_partial_resync.m64'
filename_output = main_folder+'bob100_resynced_notext.m64'

game_vars_match_exact = ['gMarioState.action',
                         'gObjectPool[113]->oFaceAngleYaw']
game_vars_match_range = {'gMarioState.pos[0]': 3.5,
                         'gMarioState.pos[1]': 40,
                         'gMarioState.pos[2]': 3.5,
                         'gObjectPool[113]->oPosX': 5.0,
                         'gObjectPool[113]->oPosZ': 5.0,
                         'gObjectPool[113]->oFaceAngleYaw': .5,#16,
                         'gMarioState.faceAngle[1]': .5,#128,
                         'gMarioState.forwardVel': .25}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 15.0,
                                  #'gMarioState.faceAngle[1]': 0.3,
                                  'gMarioState.forwardVel': 30.0
                                  }

resync_tas(game,
           filename_original, filename_toresync, filename_output,
           22493, 22994, 23837,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 0.1,
           weighted_sum_acceptable = 0.01,
           frames_back = 40,
                max_iterations = 20000)


*************************************************************
BEFORE ANNEALING
Frame 23837: fitness is 0.0
Exact matches:
gMarioState.action: wanted 536875782, got 536875782
Matches within an interval:
gMarioState.pos[0]: wanted 1972.8505859375, tolerance 3.5, got 1972.8505859375
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5869.03076171875, tolerance 3.5, got -5869.03076171875
gObjectPool[113]->oPosX: wanted 1636.0, tolerance 5.0, got 1636.0
gObjectPool[113]->oPosZ: wanted -5567.0, tolerance 5.0, got -5567.0
gObjectPool[113]->oFaceAngleYaw: wanted 24007, tolerance 0.5, got 24007
gMarioState.faceAngle[1]: wanted -8761, tolerance 0.5, got -8761
gMarioState.forwardVel: wanted 48.0, tolerance 0.25, got 48.0
Weighted approximations:
gMarioState.pos[0]: wanted 1972.8505859375, weight 15.0, got 1972.8505859375
gMarioState.pos[1]: wanted 4293.0, weight 1.0, got 4293.0
gMarioState.pos[2]: wanted -5869.03076171875, weight 15.0, got -5869.03076171875
gM

*************************************************************
BEFORE ANNEALING
Frame 23840: fitness is 2254616.578194754
Exact matches:
gMarioState.action: wanted 67109952, got 67109952
Matches within an interval:
gMarioState.pos[0]: wanted 1881.376953125, tolerance 3.5, got 1883.63916015625
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5885.78271484375, tolerance 3.5, got -5893.67333984375
gObjectPool[113]->oPosX: wanted 1636.0, tolerance 5.0, got 1636.0
gObjectPool[113]->oPosZ: wanted -5567.0, tolerance 5.0, got -5567.0
gObjectPool[113]->oFaceAngleYaw: wanted 24007, tolerance 0.5, got 24007
gMarioState.faceAngle[1]: wanted -18156, tolerance 0.5, got -18155
gMarioState.forwardVel: wanted 46.0, tolerance 0.25, got 46.0
Weighted approximations:
gMarioState.pos[0]: wanted 1881.376953125, weight 15.0, got 1883.63916015625
gMarioState.pos[1]: wanted 4293.0, weight 1.0, got 4293.0
gMarioState.pos[2]: wanted -5885.78271484375, weight 15.0, got -5893.

*************************************************************
BEFORE ANNEALING
Frame 23843: fitness is 1.259765625
Exact matches:
gMarioState.action: wanted 67109952, got 67109952
Matches within an interval:
gMarioState.pos[0]: wanted 1754.19189453125, tolerance 3.5, got 1754.17822265625
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5857.41552734375, tolerance 3.5, got -5857.34521484375
gObjectPool[113]->oPosX: wanted 1636.0, tolerance 5.0, got 1636.0
gObjectPool[113]->oPosZ: wanted -5567.0, tolerance 5.0, got -5567.0
gObjectPool[113]->oFaceAngleYaw: wanted 24007, tolerance 0.5, got 24007
gMarioState.faceAngle[1]: wanted -12012, tolerance 0.5, got -12012
gMarioState.forwardVel: wanted 43.0, tolerance 0.25, got 43.0
Weighted approximations:
gMarioState.pos[0]: wanted 1754.19189453125, weight 15.0, got 1754.17822265625
gMarioState.pos[1]: wanted 4293.0, weight 1.0, got 4293.0
gMarioState.pos[2]: wanted -5857.41552734375, weight 15.0, got -5857.34

*************************************************************
BEFORE ANNEALING
Frame 23847: fitness is 1.259765625
Exact matches:
gMarioState.action: wanted 8389719, got 8389719
Matches within an interval:
gMarioState.pos[0]: wanted 1649.50244140625, tolerance 3.5, got 1649.48876953125
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5736.63427734375, tolerance 3.5, got -5736.56396484375
gObjectPool[113]->oPosX: wanted 1636.0, tolerance 5.0, got 1636.0
gObjectPool[113]->oPosZ: wanted -5567.0, tolerance 5.0, got -5567.0
gObjectPool[113]->oFaceAngleYaw: wanted 24007, tolerance 0.5, got 24007
gMarioState.faceAngle[1]: wanted -5870, tolerance 0.5, got -5870
gMarioState.forwardVel: wanted 39.0, tolerance 0.25, got 39.0
Weighted approximations:
gMarioState.pos[0]: wanted 1649.50244140625, weight 15.0, got 1649.48876953125
gMarioState.pos[1]: wanted 4293.0, weight 1.0, got 4293.0
gMarioState.pos[2]: wanted -5736.63427734375, weight 15.0, got -5736.563964

*************************************************************
BEFORE ANNEALING
Frame 23851: fitness is 0.0
Exact matches:
gMarioState.action: wanted 899, got 899
Matches within an interval:
gMarioState.pos[0]: wanted 1574.8497314453125, tolerance 3.5, got 1574.8497314453125
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5683.9814453125, tolerance 3.5, got -5683.9814453125
gObjectPool[113]->oPosX: wanted 1636.0, tolerance 5.0, got 1636.0
gObjectPool[113]->oPosZ: wanted -5567.0, tolerance 5.0, got -5567.0
gObjectPool[113]->oFaceAngleYaw: wanted 24007, tolerance 0.5, got 24007
gMarioState.faceAngle[1]: wanted -5870, tolerance 0.5, got -5870
gMarioState.forwardVel: wanted 0.0, tolerance 0.25, got 0.0
Weighted approximations:
gMarioState.pos[0]: wanted 1574.8497314453125, weight 15.0, got 1574.8497314453125
gMarioState.pos[1]: wanted 4293.0, weight 1.0, got 4293.0
gMarioState.pos[2]: wanted -5683.9814453125, weight 15.0, got -5683.9814453125
gMarioSt

...

*************************************************************
BEFORE ANNEALING
Frame 23980: fitness is 0.2589225769042969
Exact matches:
gMarioState.action: wanted 16779394, got 16779394
Matches within an interval:
gMarioState.pos[0]: wanted 1339.8018798828125, tolerance 3.5, got 1339.7994384765625
gMarioState.pos[1]: wanted 4572.0, tolerance 40, got 4572.0
gMarioState.pos[2]: wanted -5240.3564453125, tolerance 3.5, got -5240.3466796875
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -5870
gMarioState.faceAngle[1]: wanted 147, tolerance 0.5, got 147
gMarioState.forwardVel: wanted -11.86882209777832, tolerance 0.25, got -11.866294860839844
Weighted approximations:
gMarioState.pos[0]: wanted 1339.8018798828125, weight 15.0, got 1339.7994384765625
gMarioState.pos[1]: wanted 4572.0, weight 1.0, got 4572

*************************************************************
AFTER ANNEALING
Frame 23983: fitness is 0.04414558410644531
Exact matches:
gMarioState.action: wanted 16779394, got 16779394
Matches within an interval:
gMarioState.pos[0]: wanted 1319.7930908203125, tolerance 3.5, got 1319.7901611328125
gMarioState.pos[1]: wanted 4395.0, tolerance 40, got 4395.0
gMarioState.pos[2]: wanted -5277.4873046875, tolerance 3.5, got -5277.4873046875
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -5870
gMarioState.faceAngle[1]: wanted 147, tolerance 0.5, got 147
gMarioState.forwardVel: wanted -13.438426971435547, tolerance 0.25, got -13.438420295715332
Weighted approximations:
gMarioState.pos[0]: wanted 1319.7930908203125, weight 15.0, got 1319.7901611328125
gMarioState.pos[1]: wanted 4395.0, weight 1.0, got 439

*************************************************************
AFTER ANNEALING
Frame 23986: fitness is 9.158792495727539
Exact matches:
gMarioState.action: wanted 67110008, got 67110008
Matches within an interval:
gMarioState.pos[0]: wanted 1318.9803466796875, tolerance 3.5, got 1318.8304443359375
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5315.216796875, tolerance 3.5, got -5315.2138671875
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -5870
gMarioState.faceAngle[1]: wanted 147, tolerance 0.5, got 147
gMarioState.forwardVel: wanted -15.532258033752441, tolerance 0.25, got -15.76113510131836
Weighted approximations:
gMarioState.pos[0]: wanted 1318.9803466796875, weight 15.0, got 1318.8304443359375
gMarioState.pos[1]: wanted 4293.0, weight 1.0, got 4293.0


*************************************************************
AFTER ANNEALING
Frame 23987: fitness is 1000000000.0567913
Exact matches:
gMarioState.action: wanted 67110008, got 67110008
Matches within an interval:
gMarioState.pos[0]: wanted 1298.34228515625, tolerance 3.5, got 1298.3460693359375
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5451.744140625, tolerance 3.5, got -5451.744140625
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -12014
gMarioState.faceAngle[1]: wanted 147, tolerance 0.5, got 147
gMarioState.forwardVel: wanted -15.221612930297852, tolerance 0.25, got -15.221611976623535
Weighted approximations:
gMarioState.pos[0]: wanted 1298.34228515625, weight 15.0, got 1298.3460693359375
gMarioState.pos[1]: wanted 4293.0, weight 1.0, got 4293.0
gM

KeyboardInterrupt: 

In [16]:
#add 500
game = wafel.Game('sm64_us.dll')
main_folder = 'm64s/'

filename_original = main_folder+'fight_king_twice.m64'
filename_toresync = main_folder+'fight_king_twice_partial_resync_2.m64'
filename_output = main_folder+'bob100_resynced_notext.m64'

game_vars_match_exact = ['gMarioState.action',
                         'gObjectPool[113]->oFaceAngleYaw']
game_vars_match_range = {'gMarioState.pos[0]': .01,#3.5,
                         'gMarioState.pos[1]': 40,
                         'gMarioState.pos[2]': .01,#3.5,
                         'gObjectPool[113]->oPosX': 5.0,
                         'gObjectPool[113]->oPosZ': 5.0,
                         'gObjectPool[113]->oFaceAngleYaw': .5,#16,
                         'gMarioState.faceAngle[1]': .5,#128,
                         'gMarioState.forwardVel': .25}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 15.0,
                                  #'gMarioState.faceAngle[1]': 0.3,
                                  'gMarioState.forwardVel': 30.0
                                  }

resync_tas(game,
           filename_original, filename_toresync, filename_output,
           22633, 22994, 23977,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 0.1,
           weighted_sum_acceptable = 0.01,
           frames_back = 40,
                max_iterations = 20000)


*************************************************************
BEFORE ANNEALING
Frame 23977: fitness is 194703161.81066513
Exact matches:
gMarioState.action: wanted 16779394, got 16779394
gObjectPool[113]->oFaceAngleYaw: wanted -5870, got -5870
Matches within an interval:
gMarioState.pos[0]: wanted 1358.2750244140625, tolerance 0.01, got 1358.2813720703125
gMarioState.pos[1]: wanted 4713.0, tolerance 40, got 4713.0
gMarioState.pos[2]: wanted -5207.5576171875, tolerance 0.01, got -5205.6005859375
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -5870
gMarioState.faceAngle[1]: wanted 147, tolerance 0.5, got 147
gMarioState.forwardVel: wanted -9.320931434631348, tolerance 0.25, got -9.566264152526855
Weighted approximations:
gMarioState.pos[0]: wanted 1358.2750244140625, weight 15.0, got 1358.28137207031

*************************************************************
AFTER ANNEALING
Frame 23980: fitness is 0.5446815490722656
Exact matches:
gMarioState.action: wanted 16779394, got 16779394
gObjectPool[113]->oFaceAngleYaw: wanted -5870, got -5870
Matches within an interval:
gMarioState.pos[0]: wanted 1339.8018798828125, tolerance 0.01, got 1339.7999267578125
gMarioState.pos[1]: wanted 4572.0, tolerance 40, got 4572.0
gMarioState.pos[2]: wanted -5240.3564453125, tolerance 0.01, got -5240.3603515625
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -5870
gMarioState.faceAngle[1]: wanted 147, tolerance 0.5, got 147
gMarioState.forwardVel: wanted -11.86882209777832, tolerance 0.25, got -11.853595733642578
Weighted approximations:
gMarioState.pos[0]: wanted 1339.8018798828125, weight 15.0, got 1339.79992675781

*************************************************************
BEFORE ANNEALING
Frame 23984: fitness is 0.2671623229980469
Exact matches:
gMarioState.action: wanted 16779394, got 16779394
gObjectPool[113]->oFaceAngleYaw: wanted -5870, got -5870
Matches within an interval:
gMarioState.pos[0]: wanted 1315.2354736328125, tolerance 0.01, got 1315.2301025390625
gMarioState.pos[1]: wanted 4328.0, tolerance 40, got 4328.0
gMarioState.pos[2]: wanted -5291.8642578125, tolerance 0.01, got -5291.8681640625
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -5870
gMarioState.faceAngle[1]: wanted 147, tolerance 0.5, got 147
gMarioState.forwardVel: wanted -14.438450813293457, tolerance 0.25, got -14.442717552185059
Weighted approximations:
gMarioState.pos[0]: wanted 1315.2354736328125, weight 15.0, got 1315.230102539

*************************************************************
AFTER ANNEALING
Frame 23986: fitness is 171875.39891242978
Exact matches:
gMarioState.action: wanted 67110008, got 67110008
gObjectPool[113]->oFaceAngleYaw: wanted -5870, got -5870
Matches within an interval:
gMarioState.pos[0]: wanted 1318.9803466796875, tolerance 0.01, got 1318.9730224609375
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5315.216796875, tolerance 0.01, got -5315.228515625
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -5870
gMarioState.faceAngle[1]: wanted 147, tolerance 0.5, got 147
gMarioState.forwardVel: wanted -15.532258033752441, tolerance 0.25, got -15.536033630371094
Weighted approximations:
gMarioState.pos[0]: wanted 1318.9803466796875, weight 15.0, got 1318.973022460937

*************************************************************
BEFORE ANNEALING
Frame 23988: fitness is 0.11289596557617188
Exact matches:
gMarioState.action: wanted 67110008, got 67110008
gObjectPool[113]->oFaceAngleYaw: wanted -5870, got -5870
Matches within an interval:
gMarioState.pos[0]: wanted 1298.1279296875, tolerance 0.01, got 1298.1279296875
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5467.275390625, tolerance 0.01, got -5467.271484375
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -5870
gMarioState.faceAngle[1]: wanted 147, tolerance 0.5, got 147
gMarioState.forwardVel: wanted -15.221612930297852, tolerance 0.25, got -15.219802856445312
Weighted approximations:
gMarioState.pos[0]: wanted 1298.1279296875, weight 15.0, got 1298.1279296875
gMarioSt

*************************************************************
AFTER ANNEALING
Frame 23991: fitness is 0.05859375
Exact matches:
gMarioState.action: wanted 8389719, got 8389719
gObjectPool[113]->oFaceAngleYaw: wanted -5870, got -5870
Matches within an interval:
gMarioState.pos[0]: wanted 1298.15087890625, tolerance 0.01, got 1298.15087890625
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5452.279296875, tolerance 0.01, got -5452.275390625
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -5870
gMarioState.faceAngle[1]: wanted 31, tolerance 0.5, got 31
gMarioState.forwardVel: wanted 6.998255729675293, tolerance 0.25, got 6.998255729675293
Weighted approximations:
gMarioState.pos[0]: wanted 1298.15087890625, weight 15.0, got 1298.15087890625
gMarioState.pos[1]: wa

*************************************************************
BEFORE ANNEALING
Frame 23995: fitness is 0.0
Exact matches:
gMarioState.action: wanted 899, got 899
gObjectPool[113]->oFaceAngleYaw: wanted -5870, got -5870
Matches within an interval:
gMarioState.pos[0]: wanted 1298.1787109375, tolerance 0.01, got 1298.1787109375
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5434.287109375, tolerance 0.01, got -5434.287109375
gObjectPool[113]->oPosX: wanted 1320.52490234375, tolerance 5.0, got 1320.52490234375
gObjectPool[113]->oPosZ: wanted -5306.0537109375, tolerance 5.0, got -5306.0537109375
gObjectPool[113]->oFaceAngleYaw: wanted -5870, tolerance 0.5, got -5870
gMarioState.faceAngle[1]: wanted 31, tolerance 0.5, got 31
gMarioState.forwardVel: wanted 2.998255729675293, tolerance 0.25, got 2.998255729675293
Weighted approximations:
gMarioState.pos[0]: wanted 1298.1787109375, weight 15.0, got 1298.1787109375
gMarioState.pos[1]: wanted 4293.0, weigh

*************************************************************
AFTER ANNEALING
Frame 23997: fitness is 111713991.51965141
Exact matches:
gMarioState.action: wanted 8914006, got 8914006
gObjectPool[113]->oFaceAngleYaw: wanted 31, got 31
Matches within an interval:
gMarioState.pos[0]: wanted 1298.010986328125, tolerance 0.01, got 1298.0194091796875
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5423.99755859375, tolerance 0.01, got -5425.0771484375
gObjectPool[113]->oPosX: wanted 1312.144287109375, tolerance 5.0, got 1312.144287109375
gObjectPool[113]->oPosZ: wanted -5334.0703125, tolerance 5.0, got -5334.07080078125
gObjectPool[113]->oFaceAngleYaw: wanted 31, tolerance 0.5, got 31
gMarioState.faceAngle[1]: wanted 31, tolerance 0.5, got 31
gMarioState.forwardVel: wanted 16.14998435974121, tolerance 0.25, got 14.711247444152832
Weighted approximations:
gMarioState.pos[0]: wanted 1298.010986328125, weight 15.0, got 1298.0194091796875
gMarioState.pos[

*************************************************************
BEFORE ANNEALING
Frame 23997: fitness is 113718318.44418526
Exact matches:
gMarioState.action: wanted 8914006, got 8914006
gObjectPool[113]->oFaceAngleYaw: wanted 31, got 31
Matches within an interval:
gMarioState.pos[0]: wanted 1298.010986328125, tolerance 0.01, got 1298.0010986328125
gMarioState.pos[1]: wanted 4293.0, tolerance 40, got 4293.0
gMarioState.pos[2]: wanted -5423.99755859375, tolerance 0.01, got -5425.09619140625
gObjectPool[113]->oPosX: wanted 1312.144287109375, tolerance 5.0, got 1312.144287109375
gObjectPool[113]->oPosZ: wanted -5334.0703125, tolerance 5.0, got -5334.07080078125
gObjectPool[113]->oFaceAngleYaw: wanted 31, tolerance 0.5, got 31
gMarioState.faceAngle[1]: wanted 31, tolerance 0.5, got 31
gMarioState.forwardVel: wanted 16.14998435974121, tolerance 0.25, got 14.686240196228027
Weighted approximations:
gMarioState.pos[0]: wanted 1298.010986328125, weight 15.0, got 1298.0010986328125
gMarioState.po

KeyboardInterrupt: 

In [ ]:
game = wafel.Game('sm64_jp.dll')
main_folder = 'm64s/'

filename_original = main_folder+'monkey_sync_scale_start.m64'
filename_toresync = main_folder+'monkey_sync_scale_start.m64'
filename_output = main_folder+'monkey_scale_start_resynced.m64'

game_vars_match_exact = ['gMarioState.action',
                         'gObjectPool[64]->oAction']
game_vars_match_range = {'gMarioState.pos[0]': 3.5,
                         'gMarioState.pos[1]': 40,
                         'gMarioState.pos[2]': 3.5,
                         'gObjectPool[64]->oPosX': 5.0,
                         'gObjectPool[64]->oPosZ': 5.0,
                         'gMarioState.forwardVel': .25}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 15.0,
                                  'gMarioState.faceAngle[1]': 0.3,
                                  'gMarioState.forwardVel': 30.0
                                  }

resync_tas(game,
           filename_original, filename_toresync, filename_output,
           72044, 72462, 74267,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 1.5,
           weighted_sum_acceptable = 0.01)


In [ ]:
game = wafel.Game('sm64_jp.dll')
main_folder = 'm64s/'

filename_original = main_folder+'with_longer_breathtaking.m64'
filename_toresync = main_folder+'with_longer_breathtaking.m64'
filename_output = main_folder+'monkey_fixed_goodtwirlyaw.m64'

game_vars_match_exact = ['gMarioState.action',
                         'gObjectPool[64]->oAction']
game_vars_match_range = {'gMarioState.pos[0]': 10.5,
                         'gMarioState.pos[1]': 40,
                         'gMarioState.pos[2]': 10.5,
                         'gObjectPool[64]->oPosX': 5.0,
                         'gObjectPool[64]->oPosZ': 5.0,
                         'gObjectPool[64]->oFaceAngleYaw': 512,
                         'gMarioState.forwardVel': 1.5}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 15.0,
                                  'gMarioState.faceAngle[1]': 0.3,
                                  'gMarioState.forwardVel': 30.0
                                  }
offset = 0
resync_tas(game,
           filename_original, filename_toresync, filename_output,
           72044+offset, 72462, 74268+offset,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 1.5,
           weighted_sum_acceptable = 0.01)


In [ ]:
game = wafel.Game('sm64_jp.dll')
main_folder = 'm64s/'

filename_original = main_folder+'monkey_fixed_goodtwirlyaw.m64'
filename_toresync = main_folder+'monkey_fixed_goodtwirlyaw.m64'
filename_output = main_folder+'monkey_redid_end.m64'

game_vars_match_exact = ['gMarioState.action',
                         'gObjectPool[64]->oAction']
game_vars_match_range = {'gMarioState.pos[0]': 10.5,
                         'gMarioState.pos[1]': 40,
                         'gMarioState.pos[2]': 10.5,
                         'gMarioState.forwardVel': 1.5}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 15.0,
                                  'gMarioState.faceAngle[1]': 0.3,
                                  'gMarioState.forwardVel': 30.0
                                  }
offset = 200
resync_tas(game,
           filename_original, filename_toresync, filename_output,
           72044+offset, 72462, 74268+offset,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 1.5,
           weighted_sum_acceptable = 0.01)


In [ ]:
game = wafel.Game('sm64_jp.dll')
main_folder = 'm64s/'

filename_original = main_folder+'hexed_scale_start_newrng.m64'
filename_toresync = main_folder+'hexed_scale_start_newrng.m64'
filename_output = main_folder+'monkey_fixed.m64'

game_vars_match_exact = ['gMarioState.action',
                         'gObjectPool[64]->oAction']
game_vars_match_range = {'gMarioState.pos[0]': 10.5,
                         'gMarioState.pos[1]': 40,
                         'gMarioState.pos[2]': 10.5,
                         'gObjectPool[64]->oPosX': 5.0,
                         'gObjectPool[64]->oPosZ': 5.0,
                         'gObjectPool[64]->oFaceAngleYaw': 512,
                         'gMarioState.forwardVel': 1.5}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 15.0,
                                  'gMarioState.faceAngle[1]': 0.3,
                                  'gMarioState.forwardVel': 30.0
                                  }
offset = 0
resync_tas(game,
           filename_original, filename_toresync, filename_output,
           72044+offset, 72462, 74267+offset,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 1.5,
           weighted_sum_acceptable = 0.01)


In [ ]:
# game = wafel.Game('sm64_jp.dll')
# main_folder = 'C://Users/David/Documents/Miscellaniety/Mupen/abc70/resyncing_after_wf_updates/'

# filename_original = main_folder+'wfreds_redid_misaclimb.m64'
# filename_toresync = main_folder+'workspace.m64'
# filename_output = main_folder+'auto_resynced_redsclimb.m64'

# game_vars_match_exact = ['gMarioState.action']
# game_vars_match_range = {'gMarioState.pos[0]': 1.5,
#                          'gMarioState.pos[1]': 20,
#                          'gMarioState.pos[2]': 1.5,
#                          'gMarioState.faceAngle[1]': 512,
#                          'gMarioState.forwardVel': .15}
# game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
#                                   'gMarioState.pos[1]': 1.0,
#                                   'gMarioState.pos[2]': 15.0,
#                                   'gMarioState.faceAngle[1]': 0.3,
#                                   'gMarioState.forwardVel': 30.0
#                                   }

# resync_tas(game,
#            filename_original, filename_toresync, filename_output,
#            28455, 28570, 29923,
#            game_vars_match_exact = game_vars_match_exact,
#            game_vars_match_range = game_vars_match_range,
#            game_vars_approximate_weighted = game_vars_approximate_weighted,
#            starting_temperature = 1.5)


In [ ]:
#Jong wf to bitdw resync
# game = wafel.Game('sm64_jp.dll')
# main_folder = 'D://David/Mupen/abc70/resync_wf_after_new_100tower/'

# #filename_original = main_folder+'wfreds_redid_misaclimb.m64'
# filename_original = main_folder+'WIP_TO_HAUNTED_BOOKS.m64'
# filename_toresync = main_folder+'put_in_jong_to_bitdw.m64'
# filename_output = main_folder+'resynced_jong_to_bitdw_notrngyet.m64'

# game_vars_match_exact = ['gMarioState.action']
# game_vars_match_range = {'gMarioState.pos[0]': .8,
#                          'gMarioState.pos[1]': 90,
#                          'gMarioState.pos[2]': .8,
#                          'gMarioState.faceAngle[1]': 512,
#                          'gMarioState.forwardVel': .2,
#                          'gMarioState.angleVel[1]': 128}
# game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
#                                   'gMarioState.pos[1]': 1.0,
#                                   'gMarioState.pos[2]': 15.0,
#                                   'gMarioState.faceAngle[0]': 0.3,
#                                   'gMarioState.faceAngle[1]': 0.3,
#                                   'gMarioState.forwardVel': 30.0,
#                                   'gMarioState.angleVel[1]': 1.0
#                                   }

# resync_tas(game,
#            filename_original, filename_toresync, filename_output,
#            34663, 34766, 34660,
#            game_vars_match_exact = game_vars_match_exact,
#            game_vars_match_range = game_vars_match_range,
#            game_vars_approximate_weighted = game_vars_approximate_weighted,
#            starting_temperature = 1.5,
#            weighted_sum_acceptable = 1.0)

In [ ]:
#MHMCM RESYNC
# game = wafel.Game('sm64_jp.dll')
# main_folder = 'D://David/Mupen/abc70/resync_wf_after_new_100tower/'

# #filename_original = main_folder+'wfreds_redid_misaclimb.m64'
# filename_original = main_folder+'old_wip.m64'
# filename_toresync = main_folder+'synced_position_mhmcm.m64'
# filename_output = main_folder+'resynced_mhmcm.m64'

# game_vars_match_exact = ['gMarioState.action']
# game_vars_match_range = {'gMarioState.pos[0]': .8,
#                          'gMarioState.pos[1]': 90,
#                          'gMarioState.pos[2]': .8,
#                          'gMarioState.faceAngle[1]': 512,
#                          'gMarioState.forwardVel': .2,
#                          'gMarioState.angleVel[1]': 128}
# game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
#                                   'gMarioState.pos[1]': 1.0,
#                                   'gMarioState.pos[2]': 15.0,
#                                   'gMarioState.faceAngle[0]': 0.3,
#                                   'gMarioState.faceAngle[1]': 0.3,
#                                   'gMarioState.forwardVel': 30.0,
#                                   'gMarioState.angleVel[1]': 1.0
#                                   }

# resync_tas(game,
#            filename_original, filename_toresync, filename_output,
#            58563, 59487, 55643,
#            game_vars_match_exact = game_vars_match_exact,
#            game_vars_match_range = game_vars_match_range,
#            game_vars_approximate_weighted = game_vars_approximate_weighted,
#            starting_temperature = 1.5,
#            frame_back = 45,
#            weighted_sum_acceptable = 1.0)

In [ ]:
# game = wafel.Game('sm64_jp.dll')
# main_folder = 'D://David/Mupen/abc70/resync_wf_after_new_100tower/'

# #filename_original = main_folder+'wfreds_redid_misaclimb.m64'
# filename_original = main_folder+'old_wip.m64'
# filename_toresync = main_folder+'do_cannonless_goodrng.m64'
# filename_output = main_folder+'auto_resynced_cannonless.m64'

# game_vars_match_exact = ['gMarioState.action',
#                          'gMarioState.numStars']
# game_vars_match_range = {'gMarioState.pos[0]': .8,
#                          'gMarioState.pos[1]': 90,
#                          'gMarioState.pos[2]': .8,
#                          'gMarioState.faceAngle[0]': 512,
#                          'gMarioState.faceAngle[1]': 512,
#                          'gMarioState.forwardVel': .5}
# game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
#                                   'gMarioState.pos[1]': 1.0,
#                                   'gMarioState.pos[2]': 15.0,
#                                   'gMarioState.faceAngle[1]': 0.3,
#                                   'gMarioState.forwardVel': 30.0
#                                   }

# resync_tas(game,
#            filename_original, filename_toresync, filename_output,
#            34065, 34180, 33760,
#            game_vars_match_exact = game_vars_match_exact,
#            game_vars_match_range = game_vars_match_range,
#            game_vars_approximate_weighted = game_vars_approximate_weighted,
#            starting_temperature = 1.5)


In [ ]:
game = wafel.Game('sm64_jp.dll')
main_folder = 'C://Users/David/Documents/Mupen/wafel_bruteforce/wafel_bruteforce/wafel_081_tools/m64s/'

#Try to resync ttm monkey with water upwarp
filename_original = main_folder+'breathtaking.m64'
filename_toresync = main_folder+'try_full_route.m64'
filename_output = main_folder+'100monkey_upwarp.m64'


game_vars_match_exact = []
game_vars_match_range = {'gMarioState.pos[0]': 3.0,
                         'gMarioState.pos[1]': 3.0,
                         'gMarioState.pos[2]': 3.0,
                         'gMarioState.forwardVel': .1,
                         'gMarioState.faceAngle[1]': 100}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 1.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 1.0,
                                  'gMarioState.faceAngle[1]': .05,
                                  'gMarioState.faceAngle[0]': .05,
                                  'gMarioState.forwardVel': 100.0}

resync_tas(game,
           filename_original, filename_toresync, filename_output,
           71878, 72077, 72313,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 3.5,
           weighted_sum_acceptable = 1.0,
           frames_back = 60,
           max_iterations = 8000,
           l1_diff_penalty_strength = 0.0)

In [ ]:
#Try to resync DDD, but ahead 1f?
game = wafel.Game('sm64_us.dll')

filename_original = 'C:/Users/David/Downloads/bowsersub_tjk_lag.m64'
filename_toresync = 'C:/Users/David/Downloads/bowsersub_tjk_lag_resync.m64'
filename_output = 'C:/Users/David/Downloads/attempt_save_1f_bowsersub.m64'

game_vars_match_exact = []
game_vars_match_range = {'gMarioState.pos[0]': 50.0,
                         'gMarioState.pos[1]': 50.0,
                         'gMarioState.pos[2]': 50.0}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 1.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 1.0,
                                  'gMarioState.faceAngle[1]': .001,
                                  'gMarioState.faceAngle[0]': .01}

resync_tas(game,
           filename_original, filename_toresync, filename_output,
           3690, 4380, 3690,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 0.2,
           weighted_sum_acceptable = 1.0,
           frames_back = 60,
           max_iterations = 6000,
           l1_diff_penalty_strength = .06)

In [ ]:
# game = wafel.Game('sm64_jp.dll')

# filename_original = 'm64s/old_wf100.m64'
# filename_toresync = 'm64s/new_wf100_toresync.m64'
# filename_output = 'm64s/resynced_attempt.m64'

# game_vars_match_exact = ['gMarioState.action',
#                          'gMarioState.numStars']
# game_vars_match_range = {'gMarioState.pos[0]': 1.5,
#                          'gMarioState.pos[1]': .01,
#                          'gMarioState.pos[2]': 1.5,
#                          'gMarioState.faceAngle[1]': 128,
#                          'gMarioState.forwardVel': .07}
# game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
#                                   'gMarioState.pos[1]': 1.0,
#                                   'gMarioState.pos[2]': 15.0,
#                                   'gMarioState.faceAngle[1]': 0.3,
#                                   'gMarioState.forwardVel': 45.0
#                                   }

# resync_tas(game,
#            filename_original, filename_toresync, filename_output,
#            29192, 29450, 28911,
#            game_vars_match_exact = game_vars_match_exact,
#            game_vars_match_range = game_vars_match_range,
#            game_vars_approximate_weighted = game_vars_approximate_weighted,
#            starting_temperature = 0.4)

In [ ]:
#old wf 100: idle after quickstop frame 29192, Wafel indexing
#gets star around 29430

#new wf 100: idle after quickstop frame 28911, Wafel indexing